In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from IPython.display import clear_output
from tqdm import tqdm

#Import Testing Functions 
from utils import plot_pred_distribution,apply_evaluate
from constraints import apply_process,get_markers_rocket_order_with_constraint

torch.manual_seed(42)
np.random.seed(42)


In [2]:
## Testing Function -- Loop Through Each Well in Testing Data
def run_prediction(mymodel,use_constraint=True,confidence_level=0.96,verbose=0):
    start = time.time()
    intervals = apply_process(df_tops,confidence_level,log=True)
    df_tops_pred = pd.DataFrame(pd.DataFrame(columns = ['wellName','MARCEL', 'SYLVAIN', 'CONRAD']))
    for well in tqdm(df_test_tops.index, desc=f"Processing Wells"):
        pred_m, df_wm = get_markers_rocket_order_with_constraint(well,f_mean, f_std,intervals, df_test_log, pred_column, wsize, input_variable, 
                                             s2s = True,
                                             model = mymodel, 
                                             xgb = False,
                                             rocket = None, 
                                             classifier_xgb = None, 
                                             classifier = None,
                                             constraint = use_constraint)
        if verbose:
            print(well,pred_m)
        row = {'wellName':well, 'MARCEL':pred_m[0], 'SYLVAIN':pred_m[1], 'CONRAD':pred_m[2]}
        row_df = pd.DataFrame([row])
        df_tops_pred = pd.concat([df_tops_pred, row_df], axis = 0, ignore_index = "True")
        
    ext = time.time() - start
    print(f'🍺 Total prediction time {ext}')
    df_tops_pred['wellName']  = df_tops_pred['wellName'].astype(float)
    df_tops_pred = df_tops_pred.sort_values(by = ['wellName']).reset_index().drop(['index'], axis = 1)
    df_tops_pred = df_tops_pred.set_index('wellName')
    return df_tops_pred

### LOAD Traing and Testing Data

In [3]:
#Load Prepared Training Data

X = np.load('prepared_data/X_201.npy')
y= np.load('prepared_data/y_201.npy')

# X = np.load('prepared_data/X_101.npy')
# y= np.load('prepared_data/y_101.npy')

# X = np.load('prepared_data/X_51.npy')
# y= np.load('prepared_data/y_51.npy')


In [4]:
#Load Testing Data
df_test_log = pd.read_parquet('testdata/logs_50.parquet', engine='fastparquet')
df_test_log.loc[df_test_log['GR'] < -1, 'GR' ] = -1
df_test_log.loc[df_test_log['GR'] > 400, 'GR' ] = 400

df_test_loc = pd.read_parquet('testdata/loc_50.parquet', engine='fastparquet')
df_test_loc = df_test_loc.reset_index()

df_test_log = df_test_loc.merge(df_test_log, how = 'inner', left_on = 'wellName', right_on = 'wellName')

df_test_tops = pd.read_csv('testdata/tops_50.csv')
df_test_tops = df_test_tops.set_index('wellName')
cols = ['MARCEL', 'SYLVAIN', 'CONRAD']
df_test_tops = df_test_tops[cols]

In [5]:
df_tops = pd.read_parquet('Training/tops.parquet', engine='fastparquet')
cols = ['MARCEL', 'SYLVAIN', 'CONRAD']
df_tops = df_tops[cols]
df_tops.dropna(inplace = True)
df_tops[df_tops['CONRAD'] - df_tops['SYLVAIN'] < 0] #Here we can see incorrect data

well_array = np.load('hackaton_training_well_one.npy', allow_pickle=True)
df_tops = df_tops[df_tops.index.isin(well_array[0][0])]

### Create DataLoader

In [6]:
#one hot for y 
y = np.eye(np.max(y) + 1)[y]

#nomralization for X
eps = 1e-6
f_mean = X.mean(axis=0)
f_std = X.std(axis=0) + eps  
X_norm= (X - f_mean) / f_std

#Split train & validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_norm, y, test_size=0.2, random_state=42)

#Create dataloader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  

X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32) 
y_valid_tensor = torch.tensor(y_valid, dtype=torch.long)  

dataset_train = TensorDataset(X_train_tensor, y_train_tensor)
dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=False)  

dataset_test = TensorDataset(X_valid_tensor, y_valid_tensor)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=False) 

### Training FUNCTION

In [7]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_loss = float('inf')

    def early_stop(self, validation_loss):
        improvement = self.min_loss - validation_loss
        if improvement > self.min_delta:
            self.min_loss = validation_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
            return False

In [16]:
def train(num_epochs,model,patience=2,verbose=0,lr=0.0001):
   
    start = time.time()
       
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    early_stopper = EarlyStopper(patience,min_delta= 1e-4)

    for epoch in range(num_epochs):

        #========================TRAIN===========================#
        model.train()
        for i, (input_tensor, target_tensor) in enumerate(dataloader_train):
            input_tensor = input_tensor
            target_tensor = target_tensor.float()
            optimizer.zero_grad()

            output = model(input_tensor) 
            train_loss = criterion(output, target_tensor)

            train_loss.backward()
            
            optimizer.step()

        #========================EVALUATE===========================#
        num_correct = 0
        num_samples = 0
        model.eval()

        with torch.no_grad():
            for i, (input_tensor, target_tensor) in enumerate(dataloader_test):
                
                input_tensor = input_tensor
                target_tensor = target_tensor.float()
                output = model(input_tensor) 
  
                validation_loss = criterion(output, target_tensor)

                target_classes = target_tensor.long()
                target_classes = torch.argmax(target_classes, dim=1)
      
                pred_classes = torch.argmax(output, dim=1)

                num_correct += torch.sum(pred_classes == target_classes).item()
                current_batch_size = input_tensor.size(0) 
                num_samples += current_batch_size
            
            accuracy = num_correct / num_samples
            if verbose:
                print(f'Epoch {epoch+1}/{num_epochs},   Training Loss:{train_loss.item():.5f},   Accuracy: {accuracy * 100:.2f}%,    Validation Loss: {validation_loss.item():.5f}')
            if epoch+1 > 5:
                if early_stopper.early_stop(validation_loss):  
                    print(f"Early Stop Triggered, Final Accuracy {accuracy * 100:.2f}")           
                    break

    ext = time.time() - start
    print('Total training time',ext)
    return model

### FOUR S2S Models

In [9]:
#import model

from s2s_model.LSTMA import LSTMAttn 
from s2s_model.BiGRUA import BiGRUAttn
from s2s_model.ConvgruA_simple import Attention,Encoder,Decoder,CONVGRUA_SIMPLE ## without skip connection and chanel attention
from s2s_model.ConvgruA import CONVGRUA 

#### BIGUA MODEL

In [10]:
input_size = X.shape[2]

In [12]:
#BIGUA HYPER PARAMETER
hidden_size = [64,128,256,512]


#TUNING
wsize = 201
input_variable=['GR']
pred_column = ['None','Marcel', 'Sylvain', 'Conrad']
best_optimal_tolerance = float('inf')
for i in hidden_size:
    print("="*60)
    print(f"\033[1mTesting Hidden_Size:{i}\033[0m")
    print(""*40)

    model = BiGRUAttn(input_size,i)
    print('Training model...')
    model=train(100,model,verbose=0)

    print('-'*10,'Prediction With Constraint','-'*10)
    df_tops_pred = run_prediction(model,use_constraint=True)
    optimal_tolerance,_ = apply_evaluate(df_test_tops,df_tops_pred)
    
    if optimal_tolerance < best_optimal_tolerance:
            best_optimal_tolerance = optimal_tolerance
            #Save Best Model
            torch.save(model, 'saved_model/bigu_model.pth')
            torch.save(model.state_dict(), 'saved_model/bigu_model_state.pth')
            #Save Best Configuration
            best_combination = {
                'hidden_size': i,
                'optimal_tolerance': best_optimal_tolerance
            }

    print('-'*10,'Prediction Without Constraint','-'*10)
    df_tops_pred = run_prediction(model,use_constraint=False)
    optimal_tolerance = apply_evaluate(df_test_tops,df_tops_pred)

print("="*60)
print("\033[1m🍺🍺Tuning Completed, Best Result is\033[0m")
for k,v in best_combination.items():
    print(k,v)

Testing Hidden_Size:64

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 10.89880108833313
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:08<00:00,  5.69it/s]


🍺 Total prediction time 9.612766981124878
tolerance 20, recall 1.0, mae 3.203333333333333
tolerance 15, recall 0.98, mae 3.203333333333333
tolerance 10, recall 0.9533333333333334, mae 3.203333333333333
tolerance 5, recall 0.8333333333333334, mae 3.203333333333333
Largest Error MARCEL: 16.0
Largest Error SYLVAIN: 17.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 17
---------- Prediction Without Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:11<00:00,  4.32it/s]


🍺 Total prediction time 12.506213188171387
tolerance 20, recall 1.0, mae 3.203333333333333
tolerance 15, recall 0.98, mae 3.203333333333333
tolerance 10, recall 0.9533333333333334, mae 3.203333333333333
tolerance 5, recall 0.8333333333333334, mae 3.203333333333333
Largest Error MARCEL: 16.0
Largest Error SYLVAIN: 17.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 17
Testing Hidden_Size:128

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 3.657831907272339
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


...

🍺 Total prediction time 31.577054262161255
tolerance 20, recall 0.9733333333333334, mae 3.6633333333333336
tolerance 15, recall 0.96, mae 3.6633333333333336
tolerance 10, recall 0.9266666666666666, mae 3.6633333333333336
tolerance 5, recall 0.8066666666666666, mae 3.6633333333333336
Largest Error MARCEL: 24.0
Largest Error SYLVAIN: 18.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 24
Testing Hidden_Size:512

Training model...
Early Stop Triggered, Final Accuracy 99.52
Total training time 15.89171314239502
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:45<00:00,  1.10it/s]


🍺 Total prediction time 47.03188991546631
tolerance 20, recall 0.96, mae 5.363333333333333
tolerance 15, recall 0.92, mae 5.363333333333333
tolerance 10, recall 0.86, mae 5.363333333333333
tolerance 5, recall 0.7066666666666667, mae 5.363333333333333
Largest Error MARCEL: 37.0
Largest Error SYLVAIN: 15.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 37
---------- Prediction Without Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:41<00:00,  1.20it/s]

🍺 Total prediction time 42.63044786453247
tolerance 20, recall 0.94, mae 7.983333333333333
tolerance 15, recall 0.9, mae 7.983333333333333
tolerance 10, recall 0.84, mae 7.983333333333333
tolerance 5, recall 0.6866666666666666, mae 7.983333333333333
Largest Error MARCEL: 37.0
Largest Error SYLVAIN: 15.0
Largest Error CONRAD: 156.0
🍺 Optimal Tolerance : 156
🍺🍺Tuning Completed, Best Result is
hidden_size 64
optimal_tolerance 17


#### LSTM Model 

In [13]:
input_size = X.shape[2]

In [16]:
hidden_size = [64,128,256,512]
output_dim = [50,100,150,200]

wsize = 201
input_variable=['GR']
pred_column = ['None','Marcel', 'Sylvain', 'Conrad']

best_optimal_tolerance = float('inf')
for i in hidden_size:
    for d in output_dim:
        print("="*60)
        print(f"\033[1mTesting Hidden_Size:{i} Output_Dim:{d}\033[0m")
        print(""*40)

        model = LSTMAttn(input_size,i,d)
        print('Training model...')
        model=train(100,model,verbose=0)

        print('-'*10,'Prediction With Constraint','-'*10)
        df_tops_pred = run_prediction(model,use_constraint=True)
        optimal_tolerance,_= apply_evaluate(df_test_tops,df_tops_pred)
        
        if optimal_tolerance < best_optimal_tolerance:
                best_optimal_tolerance = optimal_tolerance
                #Save Best Model
                torch.save(model, 'saved_model/lstm_model.pth')
                torch.save(model.state_dict(), 'saved_model/lstm_model_state.pth')
                #Save Best Configuration
                best_combination = {
                    'hidden_size': i,
                    'output_dim' : d,
                    'optimal_tolerance': best_optimal_tolerance
                }
print("="*60)
print("\033[1m🍺🍺Tuning Completed, Best Result is\033[0m")
for k,v in best_combination.items():
    print(k,v)

Testing Hidden_Size:64 Output_Dim:50

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 8.307817935943604
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:18<00:00,  2.67it/s]


🍺 Total prediction time 20.63109803199768
tolerance 20, recall 0.9933333333333333, mae 2.703333333333333
tolerance 15, recall 0.98, mae 2.703333333333333
tolerance 10, recall 0.96, mae 2.703333333333333
tolerance 5, recall 0.8466666666666667, mae 2.703333333333333
Largest Error MARCEL: 29.0
Largest Error SYLVAIN: 10.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 29
Testing Hidden_Size:64 Output_Dim:100

Training model...
Early Stop Triggered, Final Accuracy 99.52
Total training time 9.575665950775146
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:19<00:00,  2.52it/s]


🍺 Total prediction time 21.274296045303345
tolerance 20, recall 0.9933333333333333, mae 3.16
tolerance 15, recall 0.9933333333333333, mae 3.16
tolerance 10, recall 0.96, mae 3.16
tolerance 5, recall 0.82, mae 3.16
Largest Error MARCEL: 25.0
Largest Error SYLVAIN: 14.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 25
Testing Hidden_Size:64 Output_Dim:150

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 11.399055004119873
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


...

🍺 Total prediction time 23.0091450214386
tolerance 20, recall 1.0, mae 3.263333333333333
tolerance 15, recall 1.0, mae 3.263333333333333
tolerance 10, recall 0.9666666666666667, mae 3.263333333333333
tolerance 5, recall 0.84, mae 3.263333333333333
Largest Error MARCEL: 15.0
Largest Error SYLVAIN: 11.0
Largest Error CONRAD: 13.0
🍺 Optimal Tolerance : 15
Testing Hidden_Size:512 Output_Dim:150

Training model...
Early Stop Triggered, Final Accuracy 99.52
Total training time 19.437301874160767
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:27<00:00,  1.82it/s]


🍺 Total prediction time 29.320560216903687
tolerance 20, recall 1.0, mae 2.5966666666666667
tolerance 15, recall 0.9933333333333333, mae 2.5966666666666667
tolerance 10, recall 0.96, mae 2.5966666666666667
tolerance 5, recall 0.88, mae 2.5966666666666667
Largest Error MARCEL: 19.0
Largest Error SYLVAIN: 11.0
Largest Error CONRAD: 13.0
🍺 Optimal Tolerance : 19
Testing Hidden_Size:512 Output_Dim:200

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 17.856985092163086
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:29<00:00,  1.68it/s]

🍺 Total prediction time 30.851239919662476
tolerance 20, recall 0.9933333333333333, mae 2.9766666666666666
tolerance 15, recall 0.9933333333333333, mae 2.9766666666666666
tolerance 10, recall 0.9733333333333334, mae 2.9766666666666666
tolerance 5, recall 0.8266666666666667, mae 2.9766666666666666
Largest Error MARCEL: 27.0
Largest Error SYLVAIN: 12.0
Largest Error CONRAD: 14.0
🍺 Optimal Tolerance : 27
🍺🍺Tuning Completed, Best Result is
hidden_size 128
output_dim 200
optimal_tolerance 14


#### CONVGRUA SIMPLE 

In [10]:
input_size = X.shape[2]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [25]:
# 
hidden_size = [64,128,256]
output_size = [100,150,200]
conv_output_dim = [64,128,256]
dense_output_dim = [256,128,64]

In [30]:
wsize = 201
input_variable=['GR']
pred_column = ['None','Marcel', 'Sylvain', 'Conrad']

best_optimal_tolerance = float('inf')
for i in hidden_size:
    for o in output_size:
        for conv_dim in conv_output_dim:
            for dense_dim in dense_output_dim:
                print("="*60)
                print(f"\033[1mTesting Hidden_Size:{i} Output_Size:{o} Conv_Output_Dim:{conv_dim} Dense_Output_Dim:{dense_dim}\033[0m")
                print(""*40)
                
                # Instantiate model
                attention = Attention(i)
                encoder = Encoder(input_size, i, 1)
                decoder = Decoder(o, i, 1, attention)
                model = CONVGRUA_SIMPLE(encoder, decoder, conv_dim, dense_dim,device,num_classes=4).to(device)

                print('Training model...')
                model=train(100,model,patience=1,verbose=0)

                print('-'*10,'Prediction With Constraint','-'*10)
                df_tops_pred = run_prediction(model,use_constraint=True)
                optimal_tolerance,_ = apply_evaluate(df_test_tops,df_tops_pred)
                
                if optimal_tolerance < best_optimal_tolerance:
                        best_optimal_tolerance = optimal_tolerance
                        #Save Best Model
                        torch.save(model, 'saved_model/convgrua_simple_model.pth')
                        torch.save(model.state_dict(), 'saved_model/convgrua_simple_model_state.pth')
                        #Save Best Configuration
                        best_combination = {
                            'hidden_size': i,
                            'conv_output_dim': conv_dim,
                            'decoder_output_size' : o,
                            'dense_output_dim': dense_dim,
                            'optimal_tolerance': best_optimal_tolerance
                        }


print("="*60)
print("\033[1m🍺🍺Tuning Completed, Best Result is\033[0m")
for k,v in best_combination.items():
    print(k,v)

Testing Hidden_Size:64 Output_Size:100 Conv_Output_Dim:64 Dense_Output_Dim:256

Training model...
Early Stop Triggered, Final Accuracy 99.29
Total training time 10.875372648239136
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:41<00:00,  1.20it/s]


🍺 Total prediction time 42.461669921875
tolerance 20, recall 0.9933333333333333, mae 2.9466666666666668
tolerance 15, recall 0.9733333333333334, mae 2.9466666666666668
tolerance 10, recall 0.96, mae 2.9466666666666668
tolerance 5, recall 0.8666666666666667, mae 2.9466666666666668
Largest Error MARCEL: 23.0
Largest Error SYLVAIN: 14.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 23
Testing Hidden_Size:64 Output_Size:100 Conv_Output_Dim:64 Dense_Output_Dim:128

Training model...
Early Stop Triggered, Final Accuracy 99.29
Total training time 13.9880690574646
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [00:38<00:00,  1.29it/s]


🍺 Total prediction time 39.6936571598053
tolerance 20, recall 0.9866666666666667, mae 3.83
tolerance 15, recall 0.9666666666666667, mae 3.83
tolerance 10, recall 0.92, mae 3.83
tolerance 5, recall 0.8066666666666666, mae 3.83
Largest Error MARCEL: 30.0
Largest Error SYLVAIN: 80.0
Largest Error CONRAD: 13.0
🍺 Optimal Tolerance : 80
Testing Hidden_Size:64 Output_Size:100 Conv_Output_Dim:64 Dense_Output_Dim:64

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 18.639523029327393
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


...

🍺 Total prediction time 143.58130192756653
tolerance 20, recall 0.9866666666666667, mae 3.026666666666667
tolerance 15, recall 0.9866666666666667, mae 3.026666666666667
tolerance 10, recall 0.9733333333333334, mae 3.026666666666667
tolerance 5, recall 0.8533333333333334, mae 3.026666666666667
Largest Error MARCEL: 24.0
Largest Error SYLVAIN: 10.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 24
Testing Hidden_Size:256 Output_Size:200 Conv_Output_Dim:256 Dense_Output_Dim:128

Training model...
Early Stop Triggered, Final Accuracy 99.41
Total training time 43.86071515083313
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [02:20<00:00,  2.82s/it]


🍺 Total prediction time 141.89581489562988
tolerance 20, recall 1.0, mae 4.3133333333333335
tolerance 15, recall 0.98, mae 4.3133333333333335
tolerance 10, recall 0.9533333333333334, mae 4.3133333333333335
tolerance 5, recall 0.7133333333333334, mae 4.3133333333333335
Largest Error MARCEL: 19.5
Largest Error SYLVAIN: 18.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 20
Testing Hidden_Size:256 Output_Size:200 Conv_Output_Dim:256 Dense_Output_Dim:64

Training model...
Early Stop Triggered, Final Accuracy 99.64
Total training time 58.74982571601868
---------- Prediction With Constraint ----------
MARCEL 6212.0 6764.0
SYLVAIN 6441.0 7010.0
CONRAD 6464.0 7033.0


Processing Wells: 100%|██████████| 50/50 [02:21<00:00,  2.83s/it]

🍺 Total prediction time 142.52780604362488
tolerance 20, recall 0.9933333333333333, mae 2.796666666666667
tolerance 15, recall 0.9866666666666667, mae 2.796666666666667
tolerance 10, recall 0.9666666666666667, mae 2.796666666666667
tolerance 5, recall 0.8933333333333333, mae 2.796666666666667
Largest Error MARCEL: 22.0
Largest Error SYLVAIN: 14.0
Largest Error CONRAD: 12.0
🍺 Optimal Tolerance : 22
🍺🍺Tuning Completed, Best Result is
hidden_size 256
conv_output_dim 64
decoder_output_size 200
dense_output_dim 256
optimal_tolerance 14


#### CONVGRUA

In [ ]:
input_size = X.shape[2]
hidden_size =[64,128,256,512]




#TUNING
wsize = 201
input_variable=['GR']
pred_column = ['None','Marcel', 'Sylvain', 'Conrad']
best_optimal_tolerance = float('inf')
for i in hidden_size:
    print("="*60)
    print(f"\033[1mTesting Hidden_Size:{i}\033[0m")
    print(""*40)

    model = CONVGRUA(input_size, hidden_size)
    print('Training model...')
    model=train(100,model,verbose=0)

    print('-'*10,'Prediction With Constraint','-'*10)
    df_tops_pred = run_prediction(model,use_constraint=True)
    optimal_tolerance,_ = apply_evaluate(df_test_tops,df_tops_pred)
    
    if optimal_tolerance < best_optimal_tolerance:
            best_optimal_tolerance = optimal_tolerance
            #Save Best Model
            torch.save(model, 'saved_model/bigu_model.pth')
            torch.save(model.state_dict(), 'saved_model/bigu_model_state.pth')
            #Save Best Configuration
            best_combination = {
                'hidden_size': i,
                'optimal_tolerance': best_optimal_tolerance
            }

    print('-'*10,'Prediction Without Constraint','-'*10)
    df_tops_pred = run_prediction(model,use_constraint=False)
    optimal_tolerance = apply_evaluate(df_test_tops,df_tops_pred)

print("="*60)
print("\033[1m🍺🍺Tuning Completed, Best Result is\033[0m")
for k,v in best_combination.items():
    print(k,v)